In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("data/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
app_cat = application_df.dtypes[application_df.dtypes != "str"].index.tolist()

# Check the number of unique values in each column
application_df[app_cat].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_types = application_df.APPLICATION_TYPE.value_counts()
app_types

# Determine which values to replace if counts are less than ...?
replace_application = list(app_types[app_types < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_types = application_df.CLASSIFICATION.value_counts()
classification_types

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_types[classification_types < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

## Adding Preprocessing Step - Binning

In [6]:
# Look at ASK_AMT value counts for binning
amt_types = application_df.ASK_AMT.value_counts()

# Determine which values to bin
ask_amt_bins = list(amt_types[amt_types < 1000].index)

# Create and replace ask_amt in bins
for amt in ask_amt_bins:
    if amt <= 49999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str("5001-49999"))
    elif amt < 99999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str("50000-99999"))
    elif amt < 499999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str("100000-499999"))
    elif amt < 999999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str("500000-999999"))
    elif amt < 4999999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str("1M-5M"))
    else:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str("5M+"))
    
application_df.ASK_AMT = application_df.ASK_AMT.astype(str)

application_df.ASK_AMT.value_counts()

5000             25398
5001-49999        2947
100000-499999     2305
50000-99999       1421
1M-5M              940
500000-999999      651
5M+                637
Name: ASK_AMT, dtype: int64

In [7]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

C:\Users\mthal\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_100000-499999,ASK_AMT_1M-5M,ASK_AMT_5000,ASK_AMT_50000-99999,ASK_AMT_500000-999999,ASK_AMT_5001-49999,ASK_AMT_5M+,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(app_cat,1)
application_df.head()

C:\Users\mthal\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_100000-499999,ASK_AMT_1M-5M,ASK_AMT_5000,ASK_AMT_50000-99999,ASK_AMT_500000-999999,ASK_AMT_5001-49999,ASK_AMT_5M+,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL_0"].values
X = application_df.drop(["IS_SUCCESSFUL_0", "IS_SUCCESSFUL_1"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\mthal\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Changing Node Amounts

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               5100      
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8,161
Trainable params: 8,161
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=50)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 658us/step - loss: 0.5717 - accuracy: 0.7215
Epoch 2/100
804/804 [==============================] - 1s 662us/step - loss: 0.5551 - accuracy: 0.7300
Epoch 3/100
804/804 [==============================] - 1s 629us/step - loss: 0.5520 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 0s 621us/step - loss: 0.5483 - accuracy: 0.7326
Epoch 5/100
804/804 [==============================] - 1s 636us/step - loss: 0.5480 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 1s 648us/step - loss: 0.5465 - accuracy: 0.7354
Epoch 7/100
804/804 [==============================] - 1s 660us/step - loss: 0.5445 - accuracy: 0.7352
Epoch 8/100
804/804 [==============================] - 1s 646us/step - loss: 0.5444 - accuracy: 0.7353
Epoch 9/100
804/804 [==============================] - 1s 645us/step - loss: 0.5432 - accuracy: 0.7358
Epoch 10/100
804/804 [==============================] - 1s 638us/step - l

804/804 [==============================] - 1s 663us/step - loss: 0.5272 - accuracy: 0.7438
Epoch 78/100
804/804 [==============================] - 1s 671us/step - loss: 0.5273 - accuracy: 0.7438
Epoch 79/100
804/804 [==============================] - 1s 658us/step - loss: 0.5271 - accuracy: 0.7450
Epoch 80/100
804/804 [==============================] - 1s 639us/step - loss: 0.5269 - accuracy: 0.7447
Epoch 81/100
804/804 [==============================] - 1s 644us/step - loss: 0.5263 - accuracy: 0.7450
Epoch 82/100
804/804 [==============================] - 1s 656us/step - loss: 0.5267 - accuracy: 0.7445
Epoch 83/100
804/804 [==============================] - 1s 651us/step - loss: 0.5270 - accuracy: 0.7448
Epoch 84/100
804/804 [==============================] - 1s 640us/step - loss: 0.5269 - accuracy: 0.7453
Epoch 85/100
804/804 [==============================] - 1s 635us/step - loss: 0.5259 - accuracy: 0.7453
Epoch 86/100
804/804 [==============================] - 1s 636us/step - loss:

## Adding Additional Neurons and Hidden Layers

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               5100      
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 30)                3030      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 18,261
Trainable params: 18,261
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 730us/step - loss: 0.5677 - accuracy: 0.7247
Epoch 2/100
804/804 [==============================] - 1s 777us/step - loss: 0.5545 - accuracy: 0.7317
Epoch 3/100
804/804 [==============================] - 1s 746us/step - loss: 0.5507 - accuracy: 0.7323
Epoch 4/100
804/804 [==============================] - 1s 757us/step - loss: 0.5483 - accuracy: 0.7332
Epoch 5/100
742/804 [==========================>...] - ETA: 0s - loss: 0.5459 - accuracy: 0.7360
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 762us/step - loss: 0.5468 - accuracy: 0.7351
Epoch 6/100
804/804 [==============================] - 1s 748us/step - loss: 0.5454 - accuracy: 0.7358
Epoch 7/100
804/804 [==============================] - 1s 736us/step - loss: 0.5451 - accuracy: 0.7350
Epoch 8/100
804/804 [==============================] - 1s 724us/step - loss: 0.5439 - accuracy: 0.7374
Epoch 9/100
804/804 [=============

804/804 [==============================] - 1s 793us/step - loss: 0.5234 - accuracy: 0.7464
Epoch 63/100
804/804 [==============================] - 1s 713us/step - loss: 0.5234 - accuracy: 0.7465
Epoch 64/100
804/804 [==============================] - 1s 705us/step - loss: 0.5230 - accuracy: 0.7458
Epoch 65/100
797/804 [============================>.] - ETA: 0s - loss: 0.5234 - accuracy: 0.7453
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 711us/step - loss: 0.5230 - accuracy: 0.7456
Epoch 66/100
804/804 [==============================] - 1s 694us/step - loss: 0.5227 - accuracy: 0.7465
Epoch 67/100
804/804 [==============================] - 1s 710us/step - loss: 0.5231 - accuracy: 0.7459
Epoch 68/100
804/804 [==============================] - 1s 722us/step - loss: 0.5226 - accuracy: 0.7460
Epoch 69/100
804/804 [==============================] - 1s 691us/step - loss: 0.5219 - accuracy: 0.7463
Epoch 70/100
789/804 [=================

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                4080      
                                                                 
 dense_8 (Dense)             (None, 50)                4050      
                                                                 
 dense_9 (Dense)             (None, 30)                1530      
                                                                 
 dense_10 (Dense)            (None, 1)                 31        
                                                                 
Total params: 9,691
Trainable params: 9,691
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 670us/step - loss: 0.5720 - accuracy: 0.7177
Epoch 2/100
804/804 [==============================] - 1s 667us/step - loss: 0.5538 - accuracy: 0.7306
Epoch 3/100
804/804 [==============================] - 1s 699us/step - loss: 0.5506 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 1s 706us/step - loss: 0.5484 - accuracy: 0.7331
Epoch 5/100
804/804 [==============================] - ETA: 0s - loss: 0.5477 - accuracy: 0.7347
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 704us/step - loss: 0.5477 - accuracy: 0.7347
Epoch 6/100
804/804 [==============================] - 1s 705us/step - loss: 0.5462 - accuracy: 0.7354
Epoch 7/100
804/804 [==============================] - 1s 683us/step - loss: 0.5453 - accuracy: 0.7355
Epoch 8/100
804/804 [==============================] - 1s 666us/step - loss: 0.5447 - accuracy: 0.7364
Epoch 9/100
804/804 [=============

804/804 [==============================] - 1s 661us/step - loss: 0.5269 - accuracy: 0.7450
Epoch 63/100
804/804 [==============================] - 1s 733us/step - loss: 0.5263 - accuracy: 0.7456
Epoch 64/100
804/804 [==============================] - 1s 685us/step - loss: 0.5268 - accuracy: 0.7446
Epoch 65/100
769/804 [===========================>..] - ETA: 0s - loss: 0.5263 - accuracy: 0.7445
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 671us/step - loss: 0.5261 - accuracy: 0.7448
Epoch 66/100
804/804 [==============================] - 1s 663us/step - loss: 0.5260 - accuracy: 0.7449
Epoch 67/100
804/804 [==============================] - 1s 662us/step - loss: 0.5264 - accuracy: 0.7455
Epoch 68/100
804/804 [==============================] - 1s 658us/step - loss: 0.5261 - accuracy: 0.7451
Epoch 69/100
804/804 [==============================] - 1s 650us/step - loss: 0.5253 - accuracy: 0.7449
Epoch 70/100
762/804 [=================

## Changing activation

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 100)               5100      
                                                                 
 dense_12 (Dense)            (None, 100)               10100     
                                                                 
 dense_13 (Dense)            (None, 30)                3030      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 18,261
Trainable params: 18,261
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 718us/step - loss: 0.5681 - accuracy: 0.7224
Epoch 2/100
804/804 [==============================] - 1s 749us/step - loss: 0.5547 - accuracy: 0.7286
Epoch 3/100
804/804 [==============================] - 1s 747us/step - loss: 0.5514 - accuracy: 0.7321
Epoch 4/100
804/804 [==============================] - 1s 757us/step - loss: 0.5487 - accuracy: 0.7320
Epoch 5/100
739/804 [==========================>...] - ETA: 0s - loss: 0.5470 - accuracy: 0.7329
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 772us/step - loss: 0.5472 - accuracy: 0.7331
Epoch 6/100
804/804 [==============================] - 1s 733us/step - loss: 0.5453 - accuracy: 0.7343
Epoch 7/100
804/804 [==============================] - 1s 741us/step - loss: 0.5441 - accuracy: 0.7350
Epoch 8/100
804/804 [==============================] - 1s 755us/step - loss: 0.5430 - accuracy: 0.7348
Epoch 9/100
804/804 [=============

804/804 [==============================] - 1s 731us/step - loss: 0.5220 - accuracy: 0.7469
Epoch 63/100
804/804 [==============================] - 1s 704us/step - loss: 0.5220 - accuracy: 0.7464
Epoch 64/100
804/804 [==============================] - 1s 704us/step - loss: 0.5230 - accuracy: 0.7456
Epoch 65/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5222 - accuracy: 0.7471
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 749us/step - loss: 0.5222 - accuracy: 0.7470
Epoch 66/100
804/804 [==============================] - 1s 712us/step - loss: 0.5217 - accuracy: 0.7467
Epoch 67/100
804/804 [==============================] - 1s 711us/step - loss: 0.5221 - accuracy: 0.7465
Epoch 68/100
804/804 [==============================] - 1s 708us/step - loss: 0.5224 - accuracy: 0.7456
Epoch 69/100
804/804 [==============================] - 1s 751us/step - loss: 0.5216 - accuracy: 0.7469
Epoch 70/100
775/804 [=================

In [20]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityOptimazation.h5")